In [15]:
import copy

In [16]:
# Intersect Matrix can be changed, this is just a representation
intersect = [["1", "*", "1"],
             ["*", "2", "*"],
             ["*", "*", "1"]]

In [17]:
# intersect = [["1", "*", "1", "*"],
#              ["*", "2", "*", "*"],
#              ["*", "*", "1", "1"],
#              ["2", "*", "3", "*"]]

In [18]:
class GridCell():
  def __init__(self):
    self.max = 0 # /
    self.min = 0 # \

In [19]:
class Grid():
  def __init__(self, intersect):
    
    self.grid = list()

    for i in range(len(intersect) - 1):
      temp = list()
      for j in range(len(intersect[0]) - 1):
        grid_cell = GridCell()
        temp.append(grid_cell)
      self.grid.append(temp)
    
  def get_grid(self):
    return self.grid

In [20]:
class State():
  def __init__(self, point_intersect, intersect, grid, turn, point):
    self.grid = grid
    self.turn = turn
    self.point = point
    # self.isTerminal = len(self.grid) * len(self.grid)
    self.intersect = intersect
    self.point_intersect = copy.deepcopy(point_intersect)
  
  def __str__(self):
    empstr = ""
    for i in range(len(self.grid)):
      for j in range(len(self.grid[0])):
        if(self.grid[i][j].max == 0 and self.grid[i][j].min == 0):
          empstr += '- '
        elif(self.grid[i][j].max == 1):
          empstr += '/ '
        else:
          empstr += '\\ '
      empstr += '\n'
    empstr += '\n'
    empstr += 'Intersect Matrix: \n'
    for i in range(len(intersect)):
      for j in range(len(intersect[0])):
        empstr += intersect[i][j] + ' '
      empstr += '\n'
    empstr += '\n'
    empstr += 'Current Turn: ' + ('max' if self.turn == 'min' else 'min') + '\n'
    empstr += 'Next Turn: ' + self.turn + '\n'
    empstr += 'point: ' + str(self.point) + '\n'
    return empstr

In [21]:
class Node():
  def __init__(self, state):
    self.state = state
    self.parent = None
    self.child = []

In [22]:
class GameTree():
  
  def __init__(self, intersect):
    grid = Grid(intersect)
    grid = grid.get_grid()
    self.init_state = State(intersect , intersect, grid, 'max', 0)
    self.init = copy.deepcopy(self.init_state)
    self.start_node = Node(self.init_state)


  def generate_legal_moves(self, node):
    grid = node.state.grid
    legal_move = list()

    for i in range(len(grid)):
      for j in range(len(grid[0])):
        if(grid[i][j].max == 0 and grid[i][j].min == 0):
          legal_move.append((i,j))
    return legal_move
  
  def add_point_max(self, point_matrix, intersect, i, j): # / (i+1,j), (i, j+1) : for point
    point = 0

    if intersect[i+1][j] != '*' and intersect[i][j+1] != '*':
      num1 = int(intersect[i+1][j])
      num2 = int(intersect[i][j+1])
      
      if(num1 - 1 == 0):
        point += int(point_matrix[i+1][j])
      intersect[i+1][j] = str(num1 - 1)
      if(num2 - 1 == 0):
        point += int(point_matrix[i][j+1])
      intersect[i][j+1] = str(num2 - 1)

    else:
      if intersect[i+1][j] != '*':
        num1 = int(intersect[i+1][j])
        if(num1 - 1 == 0):
          point += int(point_matrix[i+1][j])
        intersect[i+1][j] = str(num1 - 1)
      elif intersect[i][j+1] != '*':
        num2 = int(intersect[i][j+1])
        if(num2 - 1 == 0):
          point += int(point_matrix[i][j+1])
        intersect[i][j+1] = str(num2 - 1)

    return point
  
  def add_point_min(self, point_matrix, intersect, i, j): # \  (i,j), (i+1, j+1) : for point
    point = 0

    if intersect[i][j] != '*' and intersect[i+1][j+1] != '*':
      num1 = int(intersect[i][j])
      num2 = int(intersect[i+1][j+1])
      
      if(num1 - 1 == 0):
        point += int(point_matrix[i][j])
      intersect[i][j] = str(num1 - 1)
      if(num2 - 1 == 0):
        point += int(point_matrix[i+1][j+1])
      intersect[i+1][j+1] = str(num2 - 1)

    else:
      if intersect[i][j] != '*':
        num1 = int(intersect[i][j])
        if(num1 - 1 == 0):
          point += int(point_matrix[i][j])
        intersect[i][j] = str(num1 - 1)
      elif intersect[i+1][j+1] != '*':
        num2 = int(intersect[i+1][j+1])
        if(num2 - 1 == 0):
          point += int(point_matrix[i+1][j+1])
        intersect[i+1][j+1] = str(num2 - 1)

    return point

  def state_transition(self, state, move):
    state = copy.deepcopy(state)
    i, j = move
    turn = state.turn
    point = -(state.point)
    intersect_matrix = copy.deepcopy(state.intersect)
    point_matrix = state.point_intersect

    if state.turn == "max":
      state.grid[i][j].max = 1 # / (i+1,j), (i, j+1) : for point
      turn = 'min'
      #point
      point = point + self.add_point_max(point_matrix, intersect_matrix, i, j)

    else:
      state.grid[i][j].min = 1 # \  (i,j), (i+1, j+1) : for point
      turn = 'max'  
      #point
      point = point + self.add_point_min(point_matrix, intersect_matrix, i, j)

    new_state = State(point_matrix, intersect_matrix, state.grid, turn, point)
    return new_state

  def succ(self, node):
    state = node.state
    legal_moves = self.generate_legal_moves(node)
    successors = list()

    for move in legal_moves:
      new_state = self.state_transition(state, move)
      successors.append(new_state)  

    return successors
  
  def is_terminal(self, node):
    grid = node.state.grid
    for i in range(len(grid)):
      for j in range(len(grid[0])):
        if(grid[i][j].max == 0 and grid[i][j].min == 0):
          return False 
    return True

  def create_tree_recursively(self, node):

    if(self.is_terminal(node)):
      return node
    
    for state in self.succ(node):
      new_node = Node(state)
      new_node.parent = node
      node.child.append(new_node)
    
      self.create_tree_recursively(new_node)

  def generate_search_tree(self):
    self.create_tree_recursively(self.start_node)
    return self.start_node
    
  def utility(self, node):
    return node.state.point if node.state.turn == 'min' else (-1 * node.state.point)
  
  def evaluate(self, node):
    e = 0
    if(node.state.turn == 'max'):
      e = self.max_eval(node, float('-inf'), float('inf'))
    else:
      e = self.min_eval(node, float('-inf'), float('inf'))
    return e
    
  def max_eval(self, node, alpha, beta):
    if self.is_terminal(node):
      return self.utility(node)
    for child in node.child:
      val = self.min_eval(child, alpha, beta)
      alpha = max(alpha, val)
      if(alpha >= beta):
        return alpha
    return alpha
    
  def min_eval(self, node, alpha, beta):
    if self.is_terminal(node):
      return self.utility(node)
    for child in node.child:
      val = self.max_eval(child, alpha, beta)
      beta = min(beta, val)
      if(alpha >= beta):
        return beta
    return beta



In [23]:
class Game():

  def __init__(self, intersect):
    self.GameTree = GameTree(intersect)
    self.game_tree = self.GameTree.generate_search_tree()

  def play(self):
    terminate = False
    turn = True #max
    
    isEvenGrid = False

    num = len(self.game_tree.state.grid) * len(self.game_tree.state.grid) 

    if(num % 2 == 0):
      isEvenGrid = True
    else:
      isEvenGrid = False


    player = int(input('If you want to be a start player enter 1 else 2: '))

    print("Initial Game State: ")
    print(self.game_tree.state)
    
    if(player == 1):
      #player starts
      current_node = self.game_tree
      while(not self.GameTree.is_terminal(current_node)):

        if(turn):
          print("-----------------------------------Player Turn (max)")
          print("Player Turn")
          legal_moves = self.GameTree.generate_legal_moves(current_node)
          print("Legal moves to be played: ", legal_moves)
          move = int(input('Which move you want to play: '))

          current_node = current_node.child[move]
          print("Played Move:")
          print(current_node.state)
          print("-----------------------------------Player Turn over")


          turn = False
        else:
          print("-----------------------------------AI Turn (min)")
          print("AI Turn")
          legal_moves = self.GameTree.generate_legal_moves(current_node)
          print("Legal moves to be played: ", legal_moves)

          eval_list = [self.GameTree.evaluate(c) for c in current_node.child]

          max_index = eval_list.index(min(eval_list))  #It goes for this part since the grid is even
          current_node = current_node.child[max_index]

          print("Played Move:")
          print(current_node.state)
          print("-----------------------------------AI Turn over")

          turn = True

      if(isEvenGrid):
        if(current_node.state.point < 0):
          print("Player wins the game with point: " , -current_node.state.point)
        elif(current_node.state.point > 0):
          print("Computer wins the game with point: " , current_node.state.point)
        else:
          print('Draw! Both points are 0.')
      else:
        if(current_node.state.point < 0):
          print("Computer wins the game with point: " , -current_node.state.point)
        elif(current_node.state.point > 0):
          print("Player wins the game with point: " , current_node.state.point)
        else:
          print('Draw! Both points are 0.')
    else:
      #Ai starts
      current_node = self.game_tree
      while(not self.GameTree.is_terminal(current_node)):

        if(turn):
          print("-----------------------------------AI Turn (max)")
          legal_moves = self.GameTree.generate_legal_moves(current_node)
          print("Legal moves to be played: ", legal_moves)

          eval_list = [self.GameTree.evaluate(c) for c in current_node.child]
          
          
          min_index = eval_list.index(max(eval_list))  # It goes for this part since the grid is even
          current_node = current_node.child[min_index]
          print("Played Move:")
          print(current_node.state)
          print("-----------------------------------AI Turn over")

          turn = False
        else:
          print("-----------------------------------Player Turn (min)")

          legal_moves = self.GameTree.generate_legal_moves(current_node)
          print("Legal moves to be played: ", legal_moves)
          move = int(input('Which move you want to play: '))

          current_node = current_node.child[move]
          print("Played Move:")
          print(current_node.state)
          print("-----------------------------------Player Turn over")
          turn = True
      if(isEvenGrid):
        if(current_node.state.point < 0):
          print("Computer wins the game with point: " , -current_node.state.point)
        elif(current_node.state.point > 0):
          print("Player wins the game with point: " , current_node.state.point)
        else:
          print('Draw! Both points are 0.')
      else:
        if(current_node.state.point < 0):
          print("Player wins the game with point: " , -current_node.state.point)
        elif(current_node.state.point > 0):
          print("Computer wins the game with point: " , current_node.state.point)
        else:
          print('Draw! Both points are 0.')

In [24]:
game = Game(intersect)

In [27]:
game.play()

Initial Game State: 
- - 
- - 

Intersect Matrix: 
1 * 1 
* 2 * 
* * 1 

Current Turn: min
Next Turn: max
point: 0

-----------------------------------Player Turn (max)
Player Turn
Legal moves to be played:  [(0, 0), (0, 1), (1, 0), (1, 1)]
Played Move:
/ - 
- - 

Intersect Matrix: 
1 * 1 
* 2 * 
* * 1 

Current Turn: max
Next Turn: min
point: 0

-----------------------------------Player Turn over
-----------------------------------AI Turn (min)
AI Turn
Legal moves to be played:  [(0, 1), (1, 0), (1, 1)]
